In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stories/train.txt


In [2]:
!pip install transformers
!pip install rank-bm25
!pip install torch
!pip install sentence-transformers

In [3]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [4]:
file_path = "/kaggle/input/stories/train.txt"  # Update with your actual Kaggle input path

with open(file_path, "r", encoding="utf-8") as file:
    stories = file.readlines()

stories = [story.strip() for story in stories if story.strip()]

In [5]:
print(stories[0])

The boy went to a video arcade. He played his favorite machine. His games didn't go very well. He told the owner about his experience. The owner explained that he had made the game settings harder.


In [6]:
# 2. Build BM25 Retrieval
tokenized_docs = [doc.lower().split() for doc in stories]
bm25 = BM25Okapi(tokenized_docs)

def retrieve_documents_bm25(query, top_k=3):
    query_tokens = query.lower().split()
    scores = bm25.get_scores(query_tokens)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [stories[i] for i in top_indices]



In [7]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [8]:
# def generate_answer_new(question, retrieval_fn):
#     print(question)
#     retrieved_docs = retrieval_fn(question)
#     context = " ".join(retrieved_docs)
#     print(retrieved_docs)
#     input_text = f" generate a story based on the context : {question}."
#     inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)
#     output = model.generate(**inputs, max_length=100)
#     return tokenizer.decode(output[0], skip_special_tokens=True)

In [9]:
# # 5. Comparison Example with an F1-specific question
# question = " A story nostalgia of being in a arcade "
# improved_answer = generate_answer_new( question , retrieve_documents_bm25)

In [10]:
# improved_answer

In [11]:
from transformers import pipeline

def generate_story_with_pipeline(question, retrieval_fn=None):
    # Optionally incorporate retrieval if provided
    context = ""
    if retrieval_fn:
        retrieved_docs = retrieval_fn(question)
        context = " ".join(retrieved_docs)
        prompt = f"Context: {context}\n\nWrite a nostalgic story about: {question}"
    else:
        prompt = f"Write a nostalgic story about: {question}"
    
    # Create pipeline with more control over generation
    generator = pipeline(
        "text-generation", 
        model="openai-community/gpt2",
        max_length=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3
    )
    
    # Generate the story
    result = generator(prompt, max_new_tokens=200)[0]['generated_text']
    
    # Return just the story part (removing the prompt)
    return result[len(prompt):]

# Example usage
story = generate_story_with_pipeline("A story nostalgia of being in an arcade")
print(story)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 game. I'll tell you what it was like to be playing that time!
My first childhood friend, the only person who ever met me at all when we were young and living together on our own property (in San Francisco) went through so much pain from my parents' death because they couldn't afford his funeral for us or pay him back if he died before their child-like age – but this is something else too; there are those things which will never change… They always make people cry out loud 'I've been fighting cancer!' But these memories remain with them even after one has lost everything as well as seeing some other loved ones die along side your friends'. And yet not once did anyone mention how horrible she would have felt had her mother gone down without giving birth instead of having someone take care - no matter where any health problems might otherwise go.. She'd come home screaming crying just thinking "What can happen?"




In [12]:
story

' game. I\'ll tell you what it was like to be playing that time!\nMy first childhood friend, the only person who ever met me at all when we were young and living together on our own property (in San Francisco) went through so much pain from my parents\' death because they couldn\'t afford his funeral for us or pay him back if he died before their child-like age – but this is something else too; there are those things which will never change… They always make people cry out loud \'I\'ve been fighting cancer!\' But these memories remain with them even after one has lost everything as well as seeing some other loved ones die along side your friends\'. And yet not once did anyone mention how horrible she would have felt had her mother gone down without giving birth instead of having someone take care - no matter where any health problems might otherwise go.. She\'d come home screaming crying just thinking "What can happen?"\n\n'

In [13]:
# def generate_music_prompt(question):
#     print(question)
#     input_text = f"generate a music feed it to the musicgen model to get a musicprompt that i can  context : {question}."
#     inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)
#     output = model.generate(**inputs, max_length=25)
#     return tokenizer.decode(output[0], skip_special_tokens=True)

In [14]:
# query = " A story nostalgia of being in a arcadey "
# music_prompt= generate_music_prompt(query)

In [15]:
from transformers import pipeline

def generate_music_prompt_with_pipeline(question):
    """
    Generates a descriptive music prompt that can be fed to MusicGen model
    based on the given question/theme.
    
    Args:
        question (str): The theme or concept to create music for
        
    Returns:
        str: A detailed music prompt suitable for MusicGen
    """
    print(f"Original query: {question}")
    
    # Create the pipeline with appropriate parameters
    generator = pipeline(
        "text-generation",
        model="openai-community/gpt2",  # You can use a larger model if available
        max_length=25,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2
    )
    
    # Create a detailed prompt template to get better results
    prompt = f"""Generate a detailed music prompt that describes the sound, mood, instruments, and style for: {question}
The music should capture the essence of:
"""
    
    # Generate the music description
    result = generator(prompt, max_new_tokens=75)[0]['generated_text']
    
    # Extract just the newly generated part (after the prompt)
    music_prompt = result[len(prompt):].strip()
    
    # Format the result for MusicGen
    formatted_prompt = f"A musical piece that sounds like {music_prompt}. The mood is nostalgic, with arcade game sounds and retro electronic elements."
    
    print(f"Generated music prompt: {formatted_prompt}")
    return formatted_prompt

# Example usage
query = "A story nostalgia of being in an arcade"
music_prompt = generate_music_prompt_with_pipeline(query)

Original query: A story nostalgia of being in an arcade


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=75) and `max_length`(=25) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated music prompt: A musical piece that sounds like love with another person's childhoods
A musical narrative about playing through memories while you are at home on your own. The mood is nostalgic, with arcade game sounds and retro electronic elements.


In [16]:
music_prompt

"A musical piece that sounds like love with another person's childhoods\nA musical narrative about playing through memories while you are at home on your own. The mood is nostalgic, with arcade game sounds and retro electronic elements."

In [17]:
# story_summary = improved_answer  # use the generated story
# emotion_prompt = "Summarize the emotional tone and key elements of this story to generate a music prompt:\n\n" 


# inputs_1 = tokenizer(emotion_prompt, return_tensors="pt", max_length=512, truncation=True)

# # Generate output
# output_1 = model.generate(**inputs_1, max_length=100)
# story_1 = tokenizer.decode(output_1[0], skip_special_tokens=True)

In [18]:
# story_1

In [19]:
# # Step 1: Summarize & Analyze the story's emotion and setting
# story_summary = story_1  # use the generated story
# emotion_prompt = f"Summarize the emotional tone and key elements of this story to generate a music prompt:\n\n{story_summary}"

# # Step 2: Use GPT (or your LLM) to generate a music prompt
# inputs = tokenizer(emotion_prompt, return_tensors="pt", max_length=512, truncation=True)
# output = model.generate(**inputs, max_length=25)
# music_prompt = tokenizer.decode(output[0], skip_special_tokens=True)

# print("🎵 Music Prompt:", music_prompt)


In [20]:
from diffusers import AudioLDMPipeline
import torch
import scipy

repo_id = "cvssp/audioldm-s-full-v2"
pipe = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/740M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/222M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
# audio = pipe(music_prompt + "generate music for this", num_inference_steps=10, audio_length_in_s=5.0).audios[0]

# # save the audio sample as a .wav file
# scipy.io.wavfile.write("techno2.wav", rate=16000, data=audio)

In [22]:
from diffusers import AudioLDMPipeline
import torch
import scipy

# Load the model
repo_id = "cvssp/audioldm-s-full-v2"
pipe = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Define your prompt
prompt = music_prompt + " generate music for this"

# The correct way to call the pipeline
# Just pass the prompt as the first argument
audio = pipe(
    prompt,  # First argument is the prompt text
    num_inference_steps=10,
    audio_length_in_s=5.0
).audios[0]

# Save the audio sample as a .wav file
scipy.io.wavfile.write("techno2.wav", rate=16000, data=audio)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to("cuda")


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

In [24]:
# from librosa import resample
# import numpy as np
# import soundfile as sf
# # Resample from 16kHz to 32kHz
# input_audio_32k = resample(input_audio, orig_sr=16000, target_sr=32000)

# # Now use the resampled audio
# inputs = processor(
#     audio=input_audio_32k,  # <-- Use resampled audio
#     sampling_rate=32000,    # <-- Explicitly set to 32kHz
#     text=["refined version of the input music"],
#     return_tensors="pt",
# ).to("cuda")

In [25]:
from librosa import load, resample
import soundfile as sf

# 1. Load the 16kHz audio file as a NumPy array
input_audio, sr = load("/kaggle/working/techno2.wav", sr=16000)  # sr=16000 forces correct sampling rate

# 2. Now resample to 32kHz
input_audio_32k = resample(input_audio, orig_sr=16000, target_sr=32000)

# 3. Save the resampled audio (optional)
sf.write("/kaggle/working/techno_32k.wav", input_audio_32k, 32000)

In [26]:
inputs = processor(
    audio=input_audio_32k,
    sampling_rate=32000,
    text=[f"refine the track based on {music_prompt} emotion"],
    return_tensors="pt",
).to("cuda")

output = model.generate(**inputs, max_new_tokens=256)


`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


In [27]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(output[0].cpu().numpy(), rate=sampling_rate)

In [28]:
# import soundfile as sf
# # Load the diffusion-generated audio
# input_audio, sr = sf.read("/kaggle/working/techno2.wav")

# # Process and generate continuation/refinement
# inputs = processor(
#     audio=input_audio,
#     sampling_rate=sr,
#     text=["refined version of the input music"],
#     return_tensors="pt",
# ).to("cuda")



In [29]:
# output = model.generate(**inputs, max_new_tokens=512)
# audio_refined = output.audios[0]

# # Save refined output
# sf.write("/kaggle/working/refined_techno2.wav", audio_refined, samplerate=32000)  # MusicGen uses 32kHz

In [30]:
# from IPython.display import Audio
# Audio("/kaggle/working/refined_techno2.wav")

In [31]:
# inputs = processor_music(
#     text=[story_1],
#     padding=True,
#     return_tensors="pt",
# )

# audio_values = model_music.generate(**inputs, max_new_tokens=256)

In [32]:
# from IPython.display import Audio

# sampling_rate = model_music.config.audio_encoder.sampling_rate
# Audio(audio_values[0].numpy(), rate=sampling_rate)